# [LangChain 怎麼玩？ LCEL (LangChain Expression Language) 篇，一定要認識的 LangChain 核心](https://myapollo.com.tw/blog/langchain-expression-language/#google_vignette)

## LCEL 有 2 個重要的目的。

### 統一的協定(Protocol) 
1. A unified interface: Every LCEL object implements the Runnable interface, which defines a common set of invocation methods (invoke, batch, stream, ainvoke, …). This makes it possible for chains of LCEL objects to also automatically support these invocations. That is, every chain of LCEL objects is itself an LCEL object.

LangChain 制定的協定(protocol)，每個元件例如(Prompt, ChatModel, LLM, OutputParser, Retriever, Tool 這些都是 LangChain 基本元件)都必須實作 1 個稱為 “Runnable” 的協定，這個協定至少包含以下方法（這些方法很重要，如果要將 Chain 應用變成 API 開放給他人使用，也可能需要實作以下方法）：

* invoke (支援單一輸入、單一輸出)
* batch (支援多個輸入、多個輸出)
* stream (支援有部分結果就輸出的模式)
* ainvoke (async 版本的 invoke)
* abatch (async 版本的 batch)
* astream (async 版本的 stream)

### 提供組合原型(Composition primitives)，讓 Chain 的開發變得簡易 
2. Composition primitives: LCEL provides a number of primitives that make it easy to compose chains, parallelize components, add fallbacks, dynamically configure chain internal, and more.

LCEL 提供元件組合的功能，也就是本系列文章中經常使用到的 | 運算子，例如：
```
prompt | llm
```
組合出來的 Chain 原型(primitives)分為 2 種：

* RunnableSequence
* RunnableParallel

## RunnableSequence #
RunnableSequence 會依序執行每個 Runnable , 例如 prompt | llm | output 裡的 prompt, llm, output 都是 Runnable , 它們透過 | 運算子串起來之後，就是 RunnableSequence ，執行上是依序執行，例如下列程式碼範例，可以體驗到何謂 RunnableSequence :



In [ ]:
# from langchain_community.llms import Ollama
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(model="gpt-4o-mini")

prompt = ChatPromptTemplate.from_messages([
    ("user", "{input}"),
])

prompt_output = prompt.invoke({"input": 'Hi there'})
llm_output = llm.invoke(prompt_output)
answer = StrOutputParser().invoke(llm_output)

print(llm_output)
print("=======================")
print(answer)

In [ ]:
chain = prompt | llm
res = chain.invoke({"input": 'Hi there'})
print(res)

## RunnableParallel #
RunnableParallel 則是可以同時執行 Runnable, 它的範例可以用 LangChain 的 RunnableLambda 體驗：

In [ ]:
from langchain_core.runnables import RunnableLambda

def add_one(x: int) -> int:
    return x + 1

def add_two(x: int) -> int:
    return x + 2

runnable_1 = RunnableLambda(add_one)
runnable_2 = RunnableLambda(add_two)

parallel = {"runnable_1": runnable_1, "runnable_2": runnable_2}

chain = RunnableLambda(lambda x: x) | parallel
answer = chain.invoke(1)

print(answer)

In [ ]:
parallel = {"r1": runnable_1, "r2": runnable_2}

chain = RunnableLambda(lambda x: x) | parallel
answer = chain.invoke(1)

print(answer)

## RunnableBranch 
目前為止，我們已經接觸 `Runnable`, `RunnableSequence`, `RunnableParallel`, `RunnableLambda`, 還有個變化型稱為 `RunnableBranch`。

`RunnableBranch` 可以讓我們動態決定要走哪個分支執行下 1 個 Runnable, 例如你可以透過使用者不同的輸入，而採用不同的語言模型，例如：



In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(model="gpt-4o-mini")

default_prompt = ChatPromptTemplate.from_messages([
    ("user", "{input}"),
])

python_prompt = ChatPromptTemplate.from_messages([
    ("system", "you are a full-stack developer"),
    ("user", "python {input}"),
])

default_chain = default_prompt | llm # you can customize the chain here
python_chain = python_prompt | llm  # you can customize the chain here

def route(x):
    if 'python' in x['input']:
        return python_chain
    return default_chain

chain = RunnableLambda(route)

print(type(chain))

# print(chain.invoke({"input": "python is the best"}))

llm_output = chain.invoke({"input": "python is the best"})

res = StrOutputParser().invoke(llm_output)

print(res)


### Example from LangChain
ref: https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.branch.RunnableBranch.html

In [ ]:
from langchain_core.runnables import RunnableBranch

branch = RunnableBranch(
    (lambda x: isinstance(x, str), lambda x: x.upper()),
    (lambda x: isinstance(x, int), lambda x: x + 1),
    (lambda x: isinstance(x, float), lambda x: x * 2),
    lambda x: "goodbye",
)

print(branch.invoke("hello")) # "HELLO"
print(branch.invoke(None)) # "goodbye"
print(branch.invoke(1)) # 2

## RunnablePassthrough 
```
Runnable to passthrough inputs unchanged or with additional keys.
```
如果你想對輸入做些加工的話，或者對前者 Runnable 輸出的結果做修改的話，例如修改使用者 prompt 加入關鍵詞，則可以使用 RunnablePassthrough 。

下列是修改使用者 prompt 的範例：

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("user", "{input}"),
])

chain = RunnablePassthrough.assign(input=lambda x: x['input'] + ' this is important to me.') | prompt
print(chain.invoke({"input": "python is the best."}))

## 輸入/輸出 (Input & Output Schema) 
從各個範例可以得知每個 Runnable 都有輸入與輸出，可是要怎麼知道它到底需要輸入什麼結構的資料，以及它會輸出什麼結構的資料呢？

這時可以查看 Runnable 的輸入與輸出的 schema, 這些 schema 被存在 `input_schema` 與 `output_schema` 兩個屬性中，只要是 Runnable 都可以查看其輸入與輸出結構。

### Example 1:

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("user", "{input}"),
])

In [ ]:
print(prompt.input_schema)
print(prompt.input_schema.schema())

In [ ]:
print(prompt.output_schema)
print(prompt.output_schema.schema())

### Example 2

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import SystemMessage, HumanMessage

prompt = ChatPromptTemplate.from_messages([
    SystemMessage("you are a full-stack developer"),
    # MessagesPlaceholder(variable_name='chat_history'),
    HumanMessage("python {input1}"),
])

In [ ]:
# print(prompt.input_schema)
print(prompt.input_schema.schema())
print(prompt.output_schema.schema())

In [ ]:
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate


llm = Ollama(model='llama2')
prompt = ChatPromptTemplate.from_messages([
    ("user", "{input}"),
])

chain = prompt | llm
print(chain.input_schema.schema())

## 列印 Chain 的樣子 
除了 input / output schema 的資訊， LangChain 也有提供 1 個方便的函式可以查看 chain 的長相，該方法為 <chain>.get_graph().print_ascii() 。

舉下列範例為例：

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(model="gpt-4o-mini")
prompt = ChatPromptTemplate.from_messages([
    ("user", "{input}"),
])

chain = prompt | llm
chain.get_graph().print_ascii()